# Grid search results summary

Read all the grid search results and save the optimal ones.

In [2]:
%load_ext autoreload
%autoreload 2

In [19]:
import json
import pandas as pd
import validation

## 1 Read the logs and save to a file

In [16]:
region_list = ['sweden-west', 'sweden-east', 'netherlands', 'saopaulo']
for region in region_list:
    file = f'../../results/gridsearch-n_{region}.txt'
    list_lines = []
    with open(file) as f:
        for jsonObj in f:
            line = json.loads(jsonObj)
            list_lines.append(line)
    df = pd.DataFrame(list_lines)
    with open('../../results/gridsearch.txt', 'a') as outfile:
        json.dump(df.loc[df['kl'] == min(df['kl']), :].to_dict('records')[0], outfile)
        outfile.write('\n')

## 2 Run src\py\parameters-validation.py
This step generate and save the visits using the optimal parameters on both calibration and validation datasets.

## 3 Compute KL divergence measure of the validation results

In [20]:
with open('../../results/gridsearch.txt', 'r') as f:
    for jsonObj in f:
        line = json.loads(jsonObj)
        region = line['region']
        file = f'../../results/{region}_validation_distances.csv'
        distances = pd.read_csv(file)
        kl = validation.DistanceMetrics().kullback_leibler(distances,
                                                           titles=['groundtruth', 'model'])
        line['kl-v'] = kl

        with open('../../results/summary.txt', 'a') as outfile:
            json.dump(line, outfile)
            outfile.write('\n')
